# Profit prediction using Arima,Sarima,LSTM (A comparision)
Research paper - https://ieeexplore.ieee.org/document/9964190 

dataset(2m) - https://excelbianalytics.com/wp/downloads-18-sample-csv-files-data-sets-for-testing-sales/

Necessery libraries

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim

In [ ]:
sales_data = pd.read_csv('Data\2m Sales Records.csv')

In [4]:
!pip install pandas matplotlib seaborn numpy torch
